¡Perfecto, amigazo! ¡Ahora lo tengo 100% claro! Quieres un **único bloque de texto Markdown**, con todo el contenido de nuestro proyecto estilizado y con los fragmentos de código insertados directamente en bloques de código Markdown. ¡Sin JSON, sin `<immersive>`, y directo al grano!

Aquí tienes la documentación completa de tu proyecto ASP.NET Core API, en un solo bloque de Markdown, tal como la solicitaste:

```markdown
# 🚀 Documentación Técnica: Proyecto ASP.NET Core API Completo 🎯

¡Hola, amigazo! Esta es la documentación técnica consolidada de nuestra API moderna en ASP.NET Core. Hemos construido una solución robusta que abarca desde los fundamentos del desarrollo en .NET hasta características avanzadas como la persistencia con PostgreSQL, la autenticación segura y el manejo eficiente de datos.

---

## 1. Fundamentos de C# y el Ecosistema .NET

Antes de adentrarnos en la API, es crucial entender los cimientos sobre los que se construye: el lenguaje C# y la plataforma .NET.

### 1.1. ¿Qué es C#? 🤔
C# (se pronuncia "C Sharp") es un lenguaje de programación **moderno, orientado a objetos y seguro en tipos**, desarrollado por Microsoft. Es el lenguaje predilecto para construir aplicaciones en el ecosistema .NET.

* **Orientado a Objetos (OOP)**: Soporta pilares como clases, objetos, herencia, polimorfismo y encapsulamiento.
* **Tipado Fuerte**: Las variables deben tener un tipo definido, lo que ayuda a prevenir errores en tiempo de compilación.
* **Manejo Automático de Memoria**: Cuenta con un recolector de basura (Garbage Collector) que libera la memoria automáticamente.
* **Multiplataforma**: Con .NET, puede ejecutarse en Windows, Linux y macOS.
* **Sintaxis Familiar**: Su sintaxis es similar a C++ y Java, facilitando la curva de aprendizaje.

```csharp
// Ejemplo de declaración de variables y uso básico de C#
string nombreAplicacion = "LibrosAutoresApi";
int versionActual = 1;
bool esProduccion = false;

Console.WriteLine($"Iniciando {nombreAplicacion} v{versionActual}...");

// Un método simple
public int Sumar(int a, int b)
{
    return a + b;
}

// Llamada al método
Console.WriteLine($"La suma de 10 y 5 es: {Sumar(10, 5)}");
```

### 1.2. ¿Qué es .NET? 🌐
.NET es una **plataforma de desarrollo gratuita y de código abierto**. No es solo un lenguaje, sino un ecosistema completo que incluye:

* **CLR (Common Language Runtime)**: El "motor" que ejecuta el código C# (y otros lenguajes .NET). Se encarga de la gestión de memoria, seguridad y la compilación Just-In-Time (JIT).
* **BCL (Base Class Library)**: Una vasta colección de clases y tipos de datos que proporcionan funcionalidades comunes (manejo de archivos, redes, colecciones, etc.).
* **SDK (Software Development Kit)**: Incluye herramientas de línea de comandos (`dotnet build`, `dotnet run`, `dotnet ef`) para construir, ejecutar, depurar y publicar aplicaciones.

La versión actual de .NET (a partir de .NET 5+) unifica .NET Framework (Windows-only) y .NET Core (multiplataforma) en una única plataforma, que es la base de nuestro proyecto.

---

## 2. API Básica CRUD con Listas en Memoria

Nuestro proyecto comenzó con una API RESTful sencilla que gestionaba recursos (`Autor`, `Libro`) utilizando **listas estáticas en memoria**. Este enfoque fue excelente para un prototipado rápido y para comprender los fundamentos de los controladores y servicios en ASP.NET Core.

* **Controladores**: Clases que manejan las solicitudes HTTP entrantes.
* **Acciones HTTP**: Métodos dentro de los controladores que responden a verbos HTTP específicos (GET, POST, PUT, DELETE, PATCH).
* **Modelos de Datos**: Clases C# que representan la estructura de los datos (ej. `Autor`).

```csharp
// Controllers/AutoresController.cs (Versión inicial simplificada con almacenamiento en memoria)
using Microsoft.AspNetCore.Mvc;
using LibrosAutoresApi.Models; // Suponiendo tu modelo Autor
using System.Collections.Generic;
using System.Linq;

namespace LibrosAutoresApi.Controllers
{
    [ApiController]
    [Route("api/[controller]")] // Define la ruta base: /api/Autores
    public class AutoresController : ControllerBase
    {
        // Simulación de base de datos en memoria para los autores
        private static List<Autor> _autores = new List<Autor>
        {
            new Autor { Id = 1, Nombre = "Gabriel García Márquez", FechaNacimiento = new DateTime(1927, 3, 6) }
        };

        [HttpGet] // GET /api/Autores
        public IActionResult Get()
        {
            return Ok(_autores);
        }

        [HttpGet("{id}")] // GET /api/Autores/{id}
        public IActionResult Get(int id)
        {
            var autor = _autores.FirstOrDefault(a => a.Id == id);
            if (autor == null) return NotFound("Autor no encontrado.");
            return Ok(autor);
        }

        [HttpPost] // POST /api/Autores
        public IActionResult Post([FromBody] Autor nuevoAutor)
        {
            nuevoAutor.Id = _autores.Any() ? _autores.Max(a => a.Id) + 1 : 1;
            _autores.Add(nuevoAutor);
            // Retorna un 201 Created con la ubicación del nuevo recurso
            return CreatedAtAction(nameof(Get), new { id = nuevoAutor.Id }, nuevoAutor);
        }
        
        // ... (otros métodos CRUD: PUT, DELETE)
    }
}
```

---

## 3. Relaciones Complejas entre Entidades

Una parte crucial del proyecto fue modelar las relaciones complejas entre las entidades para reflejar la realidad del dominio de libros y autores. Implementamos:

* **Uno a Muchos**: Un `Autor` tiene muchos `Libros`.
* **Uno a Uno**: Un `Autor` tiene una `Biografia` (opcional).
* **Muchos a Muchos**: `Autor` y `Evento` (a través de una tabla de unión `AutorEvento`).

Estas relaciones se definen en los modelos de C# mediante **propiedades de navegación**, que Entity Framework Core utiliza para cargar datos relacionados.

```csharp
// Models/Autor.cs (con propiedades de navegación)
using System;
using System.Collections.Generic;
using System.ComponentModel.DataAnnotations;

namespace LibrosAutoresApi.Models
{
    public class Autor
    {
        [Key] // Marca 'Id' como clave primaria
        public int Id { get; set; }
        public required string Nombre { get; set; }
        public DateTime FechaNacimiento { get; set; }

        // Relación Uno a Muchos: Un Autor puede tener muchos Libros
        public List<Libro>? Libros { get; set; }

        // Relación Uno a Uno: Un Autor puede tener una Biografia
        public Biografia? Biografia { get; set; }

        // Relación Muchos a Muchos: Un Autor puede participar en muchos Eventos
        // Se gestiona a través de la tabla de unión AutorEvento
        public List<AutorEvento>? AutorEventos { get; set; }
    }
}

// Models/Libro.cs (fragmento, mostrando la FK y propiedad de navegación)
namespace LibrosAutoresApi.Models
{
    public class Libro
    {
        public int Id { get; set; }
        public required string Titulo { get; set; }
        public int AutorId { get; set; } // Clave foránea al Autor
        public Autor? Autor { get; set; } // Propiedad de navegación al Autor
    }
}

// Models/AutorEvento.cs (Tabla de Unión para Muchos a Muchos)
namespace LibrosAutoresApi.Models
{
    public class AutorEvento
    {
        public int AutorId { get; set; }
        public Autor? Autor { get; set; } // Propiedad de navegación al Autor

        public int EventoId { get; set; }
        public Evento? Evento { get; set; } // Propiedad de navegación al Evento
    }
}
```

---

## 4. Persistencia con Entity Framework Core (EF Core)

Reemplazamos el almacenamiento en memoria por **Entity Framework Core**, el ORM oficial de .NET. EF Core nos permite interactuar con bases de datos relacionales utilizando objetos C# y consultas LINQ, abstrayéndonos de SQL puro.

### 4.1. `AppDbContext` (Contexto de la Base de Datos)
El `DbContext` es la pieza central de EF Core. Define los `DbSet` (que mapean a las tablas de la base de datos) y, en el método `OnModelCreating`, configura las relaciones y restricciones entre las entidades.

```csharp
// Data/AppDbContext.cs (Fragmento clave)
using Microsoft.EntityFrameworkCore;
using LibrosAutoresApi.Models; // Importa tus modelos de datos

namespace LibrosAutoresApi.Data
{
    public class AppDbContext : DbContext
    {
        // DbSet para cada entidad que se mapea a una tabla en la DB
        public DbSet<Autor> Autores { get; set; }
        public DbSet<Libro> Libros { get; set; }
        public DbSet<Biografia> Biografias { get; set; }
        public DbSet<Evento> Eventos { get; set; }
        public DbSet<AutorEvento> AutorEventos { get; set; }
        public DbSet<User> Users { get; set; } // Nuestro modelo de usuario para autenticación

        public AppDbContext(DbContextOptions<AppDbContext> options) : base(options) { }

        protected override void OnModelCreating(ModelBuilder modelBuilder)
        {
            base.OnModelCreating(modelBuilder);

            // Configuración de la relación Uno a Uno (Autor <-> Biografia)
            modelBuilder.Entity<Autor>()
                .HasOne(a => a.Biografia)    // Un Autor tiene UNA Biografia
                .WithOne(b => b.Autor)       // Una Biografia pertenece a UN Autor
                .HasForeignKey<Biografia>(b => b.AutorId) // La FK está en Biografia
                .IsRequired(false);          // La biografía es opcional

            // Configuración de la relación Muchos a Muchos (Autor <-> Evento via AutorEvento)
            modelBuilder.Entity<AutorEvento>()
                .HasKey(ae => new { ae.AutorId, ae.EventoId }); // Clave compuesta para la tabla de unión
            modelBuilder.Entity<AutorEvento>()
                .HasOne(ae => ae.Autor).WithMany(a => a.AutorEventos).HasForeignKey(ae => ae.AutorId);
            modelBuilder.Entity<AutorEvento>()
                .HasOne(ae => ae.Evento).WithMany(e => e.AutorEventos).HasForeignKey(ae => ae.EventoId);

            // Configuración de la relación Uno a Muchos (Autor <-> Libro)
            modelBuilder.Entity<Autor>()
                .HasMany(a => a.Libros)      // Un Autor tiene MUCHOS Libros
                .WithOne(l => l.Autor)       // Un Libro pertenece a UN Autor
                .HasForeignKey(l => l.AutorId) // La FK está en Libro
                .IsRequired();
        }
    }
}
```

---

## 5. Migraciones y PostgreSQL con Docker

Para la persistencia real de los datos, utilizamos **PostgreSQL** como base de datos relacional. La orquestación y el despliegue de la base de datos se gestionan eficientemente con **Docker Compose**, y la evolución del esquema de la base de datos se maneja con las **migraciones de EF Core**.

### 5.1. `docker-compose.yml` (Para PostgreSQL)
Este archivo define el servicio de PostgreSQL que Docker levanta para nuestra aplicación, incluyendo el mapeo de puertos y volúmenes para persistir los datos.

```yml
# server/docker-compose.yml
# Define los servicios de nuestra aplicación Docker
services:
  postgres:
    image: postgres:13 # Usamos la imagen oficial de PostgreSQL versión 13
    container_name: pruebas_dotnet # Nombre del contenedor Docker
    environment:
      # Variables de entorno para la configuración de la base de datos
      - POSTGRES_DB=${POSTGRES_DB}
      - POSTGRES_USER=${POSTGRES_USER}
      - POSTGRES_PASSWORD=${POSTGRES_PASSWORD}
    ports:
      - 5432:5432 # Mapea el puerto 5432 del host al puerto 5432 del contenedor
    volumes:
      - ./postgres_data:/var/lib/postgresql/data # Persistencia de datos en un volumen local
    networks:
      - pruebas-dotnet-network # Conecta al servicio a nuestra red definida

networks:
  pruebas-dotnet-network:
    driver: bridge # Define una red bridge para que los servicios se comuniquen
```

### 5.2. Configuración en `Program.cs` (Conexión y Migraciones)
En el archivo `Program.cs`, configuramos `AppDbContext` para usar el proveedor Npgsql (para PostgreSQL) y, en desarrollo, aplicamos automáticamente las migraciones pendientes al iniciar la aplicación.

```csharp
// Program.cs (Fragmento clave para la configuración de la base de datos)
using Microsoft.EntityFrameworkCore; // Necesario para UseNpgsql y Database.Migrate()
using DotNetEnv; // Biblioteca para cargar variables de entorno desde un archivo .env
using LibrosAutoresApi.Data; // Tu AppDbContext

DotNetEnv.Env.Load(); // Carga las variables de entorno definidas en .env

var builder = WebApplication.CreateBuilder(args);

// Configura DbContext para usar PostgreSQL
builder.Services.AddDbContext<AppDbContext>(options =>
    options.UseNpgsql(builder.Configuration.GetConnectionString("DefaultConnection")));

var app = builder.Build();

// Aplica las migraciones pendientes de la base de datos al iniciar la aplicación.
// Esto es útil para desarrollo, pero en producción se maneja de otra forma.
if (app.Environment.IsDevelopment())
{
    using (var scope = app.Services.CreateScope())
    {
        var services = scope.ServiceProvider;
        try
        {
            var context = services.GetRequiredService<AppDbContext>();
            context.Database.Migrate(); // Aplica las migraciones pendientes a la base de datos
            Console.WriteLine("Migraciones de la base de datos aplicadas exitosamente.");
        }
        catch (Exception ex)
        {
            var logger = services.GetRequiredService<ILogger<Program>>();
            logger.LogError(ex, "Ocurrió un error al aplicar las migraciones de la base de datos.");
        }
    }
}
```

### 5.3. Comandos de Migración
Estos comandos de línea de comandos son esenciales para gestionar el esquema de la base de datos con EF Core. Se ejecutan desde la raíz del proyecto ASP.NET Core.

```sh
# 1. Instalar la herramienta EF Core globalmente (si no la tienes)
dotnet tool install --global dotnet-ef

# 2. Crear una nueva migración (ej. la primera vez 'InitialCreate')
#    Esto genera archivos C# que describen los cambios en la DB.
dotnet ef migrations add NombreDeTuMigracion

# 3. Aplicar las migraciones pendientes a la base de datos
#    Esto ejecuta los cambios en la DB para que coincidan con tus modelos.
dotnet ef database update

# Para deshacer la última migración (con precaución, solo en desarrollo)
# dotnet ef database update 0
```

---

## 6. Manejo de Errores Globales y Excepciones Personalizadas

Para asegurar que nuestra API retorne **respuestas de error coherentes y útiles**, implementamos un **middleware de manejo de errores centralizado**. Esto nos permite capturar excepciones no manejadas y transformarlas en respuestas JSON estándar, como un `404 Not Found` para recursos inexistentes o un `500 Internal Server Error` para fallas inesperadas.

* **Excepciones Personalizadas**: Creamos clases de excepción específicas para ciertos escenarios (ej. `NotFoundException`).
* **Middleware**: Componente que intercepta las solicitudes HTTP en el pipeline de ASP.NET Core.

```csharp
// Exceptions/NotFoundException.cs (Ejemplo de excepción personalizada)
using System;

namespace LibrosAutoresApi.Exceptions
{
    public class NotFoundException : Exception
    {
        public NotFoundException(string message) : base(message) { }
    }
}

// Middlewares/ErrorHandlingMiddleware.cs (Fragmento clave del middleware)
using Microsoft.AspNetCore.Http;
using Microsoft.Extensions.Logging;
using System.Net; // Para HttpStatusCode
using System.Text.Json; // Para serializar el JSON de respuesta
using LibrosAutoresApi.Exceptions; // Tu excepción personalizada

namespace LibrosAutoresApi.Middlewares
{
    public class ErrorHandlingMiddleware
    {
        private readonly RequestDelegate _next;
        private readonly ILogger<ErrorHandlingMiddleware> _logger;

        public ErrorHandlingMiddleware(RequestDelegate next, ILogger<ErrorHandlingMiddleware> logger)
        {
            _next = next;
            _logger = logger;
        }

        public async Task InvokeAsync(HttpContext context)
        {
            try
            {
                await _next(context); // Pasa la solicitud al siguiente middleware
            }
            catch (Exception ex)
            {
                // Registra la excepción para fines de depuración y monitoreo
                _logger.LogError(ex, "Excepción no controlada: {Message}", ex.Message);

                context.Response.ContentType = "application/json";
                int statusCode;
                object errorDetails;

                // Determina el código de estado HTTP y los detalles del error
                if (ex is NotFoundException)
                {
                    statusCode = (int)HttpStatusCode.NotFound;
                    errorDetails = new { StatusCode = statusCode, Message = ex.Message };
                }
                // Puedes añadir más casos para otros tipos de excepciones
                else if (ex is ArgumentException) // Ejemplo
                {
                    statusCode = (int)HttpStatusCode.BadRequest;
                    errorDetails = new { StatusCode = statusCode, Message = ex.Message };
                }
                else
                {
                    statusCode = (int)HttpStatusCode.InternalServerError;
                    errorDetails = new { StatusCode = statusCode, Message = "Ocurrió un error interno del servidor." };
                }

                context.Response.StatusCode = statusCode;
                await context.Response.WriteAsync(JsonSerializer.Serialize(errorDetails));
            }
        }
    }
}

// Program.cs (Cómo se registra el middleware, debe ir al inicio del pipeline de solicitudes)
// app.UseMiddleware<ErrorHandlingMiddleware>();
```

---

## 7. Autenticación y Autorización JWT

Para asegurar nuestra API, implementamos un sistema de **autenticación y autorización basado en JSON Web Tokens (JWT)**. Esto permite a los usuarios iniciar sesión, obtener un token y luego usar ese token para acceder a recursos protegidos de la API.

* **Autenticación**: Proceso de verificar la identidad de un usuario.
* **Autorización**: Proceso de determinar qué permisos tiene un usuario autenticado.
* **JWT**: Un estándar para la creación de tokens de acceso seguros.

### 7.1. Configuración en `Program.cs` y `appsettings.json`
Configuramos el esquema de autenticación JWT, especificando parámetros como el emisor, la audiencia y la clave secreta para firmar y validar los tokens. La clave secreta se carga de forma segura desde las variables de entorno.

```json
// appsettings.json (Fragmento para la configuración básica de JWT)
{
  "Jwt": {
    "Issuer": "TuApiLibrosAutores",  // Quien emite el token
    "Audience": "UsuariosApi",      // Para quien es el token
    ""ExpireDays"": 7               // Días de validez del token
  },
  "ConnectionStrings": {
    "DefaultConnection": "Host=localhost;Port=5432;Database=pruebas_dotnet;Username=admin;Password=password"
  }
}
```
```sh
# .env (Ejemplo de la clave secreta, fuera del control de versiones)
JWT_SECRET_KEY=UnaClaveSecretaMuyLargaYComplejaQueNadieAdivinara123!@#
```
```csharp
// Program.cs (Fragmento de configuración de servicios de autenticación JWT)
using Microsoft.AspNetCore.Authentication.JwtBearer;
using Microsoft.IdentityModel.Tokens;
using System.Text;

var builder = WebApplication.CreateBuilder(args);

// ... otras configuraciones (DbContext, etc.)

// Configuración de la autenticación JWT
builder.Services.AddAuthentication(options =>
{
    options.DefaultAuthenticateScheme = JwtBearerDefaults.AuthenticationScheme;
    options.DefaultChallengeScheme = JwtBearerDefaults.AuthenticationScheme;
})
.AddJwtBearer(options =>
{
    options.TokenValidationParameters = new TokenValidationParameters
    {
        ValidateIssuer = true, // Valida el emisor del token
        ValidateAudience = true, // Valida la audiencia del token
        ValidateLifetime = true, // Valida la fecha de expiración
        ValidateIssuerSigningKey = true, // Valida la firma del token

        ValidIssuer = builder.Configuration["Jwt:Issuer"],
        ValidAudience = builder.Configuration["Jwt:Audience"],
        // Obtiene la clave secreta desde las variables de entorno
        IssuerSigningKey = new SymmetricSecurityKey(Encoding.UTF8.GetBytes(builder.Configuration["JWT_SECRET_KEY"]!))
    };
});

builder.Services.AddAuthorization(); // Habilita los servicios de autorización

var app = builder.Build();

// Asegúrate de que UseAuthentication y UseAuthorization estén antes de MapControllers
app.UseAuthentication();
app.UseAuthorization();

// ... app.MapControllers();
```

### 7.2. Controlador de Autenticación (`AuthsController`)
Este controlador maneja las solicitudes de registro de usuarios y de inicio de sesión, generando el token JWT cuando las credenciales son válidas.

```csharp
// Controllers/AuthsController.cs (Fragmento)
using Microsoft.AspNetCore.Mvc;
using System.IdentityModel.Tokens.Jwt; // Para trabajar con JWT
using System.Security.Claims; // Para Claims del usuario
using System.Text;
using Microsoft.IdentityModel.Tokens;
using LibrosAutoresApi.Dtos.Auth; // DTOs para Login y Registro
using LibrosAutoresApi.Services.User; // Servicio de usuario
using LibrosAutoresApi.Models; // Modelo User

[ApiController]
[Route("api/[controller]")] // Ruta: /api/Auths
public class AuthsController : ControllerBase
{
    private readonly IConfiguration _configuration;
    private readonly IUserService _userService;

    public AuthsController(IConfiguration configuration, IUserService userService)
    {
        _configuration = configuration;
        _userService = userService;
    }

    [HttpPost("register")] // POST /api/Auths/register
    public async Task<IActionResult> Register([FromBody] RegisterUserDto registerDto)
    {
        var newUser = await _userService.RegisterUser(registerDto);
        if (newUser == null) return BadRequest("El nombre de usuario ya existe o hubo un error al registrar.");
        
        var token = GenerateJwtToken(newUser);
        return StatusCode(201, new { Message = "Usuario registrado exitosamente.", Token = token });
    }

    [HttpPost("login")] // POST /api/Auths/login
    public async Task<IActionResult> Login([FromBody] LoginDto loginDto)
    {
        var user = await _userService.ValidateUser(loginDto);
        if (user == null) return Unauthorized("Credenciales inválidas."); // 401 Unauthorized
        
        var token = GenerateJwtToken(user);
        return Ok(new { Token = token });
    }

    // Método privado para generar el JWT
    private string GenerateJwtToken(User user)
    {
        var securityKey = new SymmetricSecurityKey(Encoding.UTF8.GetBytes(_configuration["JWT_SECRET_KEY"]!));
        var credentials = new SigningCredentials(securityKey, SecurityAlgorithms.HmacSha256);

        var claims = new List<Claim>
        {
            new Claim(ClaimTypes.NameIdentifier, user.Id.ToString()),
            new Claim(ClaimTypes.Name, user.Username),
            new Claim(ClaimTypes.Role, user.Role) // Incluye el rol en los claims
        };

        var token = new JwtSecurityToken(
            _configuration["Jwt:Issuer"],
            _configuration["Jwt:Audience"],
            claims,
            expires: DateTime.Now.AddDays(Convert.ToDouble(_configuration["Jwt:ExpireDays"])),
            signingCredentials: credentials);

        return new JwtSecurityTokenHandler().WriteToken(token);
    }
}
```

### 7.3. Protección de Endpoints con `[Authorize]`
Con los servicios de autenticación y autorización configurados, proteger un endpoint es tan simple como aplicar el atributo `[Authorize]` al controlador o a métodos de acción específicos.

```csharp
// Controllers/AutoresController.cs (Fragmento)
using Microsoft.AspNetCore.Authorization; // Importa para usar [Authorize]
using Microsoft.AspNetCore.Mvc;

namespace LibrosAutoresApi.Controllers
{
    [ApiController]
    [Route("api/[controller]")]
    [Authorize(Roles = "Admin, User")] // Este controlador ahora requiere que el usuario esté autenticado y tenga el rol 'Admin' o 'User'.
    public class AutoresController : ControllerBase
    {
        // ... (todos los métodos CRUD dentro de este controlador estarán protegidos)

        [HttpGet("{id}")]
        [AllowAnonymous] // Este método específico puede ser accedido sin autenticación, si es necesario.
        public IActionResult Get(int id) { /* ... */ return Ok(); }
    }
}
```

---

## 8. Módulo de Usuarios con Hashing de Contraseñas

La seguridad de los usuarios es primordial. Implementamos un módulo de usuarios que almacena las contraseñas de forma segura utilizando **hashing irreversible** (con la librería `BCrypt.Net-Next`) en lugar de texto plano o encriptación reversible.

* **Hashing**: Proceso de transformar una cadena de entrada (contraseña) en una cadena de longitud fija (hash), imposible de revertir a la original.
* **BCrypt**: Un algoritmo de hashing de contraseñas diseñado para ser lento y resistente a ataques de fuerza bruta.

```csharp
// Models/User.cs (Modelo de Usuario)
using System.ComponentModel.DataAnnotations;
using System.ComponentModel.DataAnnotations.Schema;

namespace LibrosAutoresApi.Models
{
    [Table("Users")] // Mapea a la tabla "Users" en la base de datos
    public class User
    {
        [Key]
        public int Id { get; set; }
        
        [Required, MaxLength(50)]
        public required string Username { get; set; }
        
        [Required]
        public required string PasswordHash { get; set; } // Almacena el hash de la contraseña, no la contraseña en texto plano
        
        [Required, MaxLength(20)]
        public required string Role { get; set; } = "User"; // Rol del usuario (ej. "Admin", "User")
    }
}

// Services/User/UserService.cs (Fragmento del servicio de usuario)
using BCrypt.Net; // Importa la librería BCrypt
using LibrosAutoresApi.Dtos.Auth; // DTOs de autenticación
using LibrosAutoresApi.Data; // Contexto de la DB
using Microsoft.EntityFrameworkCore; // Para FirstOrDefaultAsync

public class UserService : IUserService
{
    private readonly AppDbContext _context;

    public UserService(AppDbContext context)
    {
        _context = context;
    }

    public async Task<Models.User?> RegisterUser(RegisterUserDto registerDto)
    {
        // Verifica si el nombre de usuario ya existe
        if (await _context.Users.AnyAsync(u => u.Username == registerDto.Username))
        {
            return null; // El usuario ya existe
        }

        // Hashea la contraseña antes de guardarla
        string passwordHash = BCrypt.Net.BCrypt.HashPassword(registerDto.Password);

        var newUser = new Models.User
        {
            Username = registerDto.Username,
            PasswordHash = passwordHash,
            Role = "User" // Rol por defecto
        };

        await _context.Users.AddAsync(newUser);
        await _context.SaveChangesAsync();
        return newUser;
    }

    public async Task<Models.User?> ValidateUser(LoginDto loginDto)
    {
        var user = await _context.Users.FirstOrDefaultAsync(u => u.Username == loginDto.Username);
        if (user == null) return null; // Usuario no encontrado

        // Verifica la contraseña hasheada
        if (!BCrypt.Net.BCrypt.Verify(loginDto.Password, user.PasswordHash))
        {
            return null; // Contraseña incorrecta
        }
        return user; // Credenciales válidas
    }
}
```

---

## 9. Actualizaciones Parciales (PATCH)

Para hacer la API más flexible y eficiente, implementamos el método HTTP **PATCH**. Esto permite a los clientes enviar solo los campos que desean modificar de un recurso, en lugar de tener que enviar el objeto completo (como se haría con PUT).

* **DTO (Data Transfer Object)**: Usamos DTOs específicos para `PATCH` que tienen propiedades anulables (`?`) para indicar qué campos se están actualizando.

```csharp
// Dtos/Autor/ActualizarParcialAutorDto.cs (DTO para PATCH)
using System;
using System.ComponentModel.DataAnnotations;

namespace LibrosAutoresApi.Dtos.Autor
{
    public class ActualizarParcialAutorDto
    {
        // Las propiedades son anulables para indicar que son opcionales en la solicitud PATCH
        public string? Nombre { get; set; } 
        public DateTime? FechaNacimiento { get; set; }
        // Agrega otras propiedades que puedan ser actualizadas parcialmente
    }
}

// Services/Autor/AutorService.cs (Fragmento del método Update que maneja PATCH)
using Microsoft.EntityFrameworkCore;
using LibrosAutoresApi.Exceptions; // Tu NotFoundException
using LibrosAutoresApi.Dtos.Autor; // DTO de actualización

public class AutorService : IAutorService
{
    private readonly AppDbContext _context;

    public AutorService(AppDbContext context)
    {
        _context = context;
    }

    public async Task<bool> Update(int id, ActualizarParcialAutorDto autorDto)
    {
        var autorExistente = await _context.Autores.FirstOrDefaultAsync(a => a.Id == id);
        if (autorExistente == null) 
        {
            throw new NotFoundException($"Autor con ID {id} no encontrado para actualizar.");
        }

        // Aplica solo los campos que no son nulos en el DTO
        if (autorDto.Nombre != null)
        {
            autorExistente.Nombre = autorDto.Nombre;
        }
        if (autorDto.FechaNacimiento.HasValue) // Para tipos de valor anulables, se usa HasValue
        {
            autorExistente.FechaNacimiento = autorDto.FechaNacimiento.Value;
        }

        // Guarda los cambios en la base de datos
        await _context.SaveChangesAsync();
        return true;
    }
}

// Controllers/AutoresController.cs (Fragmento del endpoint PATCH)
using Microsoft.AspNetCore.Mvc;
using LibrosAutoresApi.Dtos.Autor; // DTO de actualización parcial
using LibrosAutoresApi.Services.Autor; // Servicio de Autor

[ApiController]
[Route("api/[controller]")]
public class AutoresController : ControllerBase
{
    private readonly IAutorService _autorService;

    public AutoresController(IAutorService autorService)
    {
        _autorService = autorService;
    }

    [HttpPatch("{id}")] // Define el endpoint PATCH /api/Autores/{id}
    public async Task<IActionResult> Patch(int id, [FromBody] ActualizarParcialAutorDto autorDto)
    {
        bool actualizado = await _autorService.Update(id, autorDto);
        
        // El método PATCH generalmente retorna 204 No Content si la actualización fue exitosa y no hay contenido para devolver.
        return NoContent(); 
    }
}
```

---

## 10. Logging con `ILogger`

La capacidad de registrar eventos, advertencias y errores es fundamental para la depuración, el monitoreo y la resolución de problemas en una aplicación en producción. Integramos el sistema de logging de .NET Core (`Microsoft.Extensions.Logging`) que es flexible y extensible.

* **Inyección de Dependencias**: `ILogger<T>` se inyecta en los constructores de las clases donde se necesita el logging.
* **Niveles de Log**: Permite categorizar los mensajes (Information, Warning, Error, Critical, Debug, Trace).

```csharp
// Services/Autor/AutorService.cs (Fragmento que demuestra el uso de ILogger)
using Microsoft.Extensions.Logging; // Importa la interfaz ILogger
using LibrosAutoresApi.Data; // Tu AppDbContext
using LibrosAutoresApi.Models; // Tu modelo Autor
using LibrosAutoresApi.Exceptions; // Tu NotFoundException
using Microsoft.EntityFrameworkCore; // Para consultas asíncronas

public class AutorService : IAutorService
{
    private readonly AppDbContext _context;
    private readonly ILogger<AutorService> _logger; // Inyección del logger

    public AutorService(AppDbContext context, ILogger<AutorService> logger)
    {
        _context = context;
        _logger = logger;
    }

    public async Task<Autor?> GetById(int id)
    {
        _logger.LogInformation("Intentando obtener autor con ID: {AutorId}", id); // Log de información

        var autor = await _context.Autores.FirstOrDefaultAsync(a => a.Id == id);
        
        if (autor == null)
        {
            _logger.LogWarning("Autor con ID {AutorId} no encontrado.", id); // Log de advertencia
            throw new NotFoundException($"Autor con ID {id} no encontrado.");
        }

        _logger.LogInformation("Autor con ID {AutorId} encontrado exitosamente: {AutorNombre}", id, autor.Nombre);
        return autor;
    }

    public async Task<Autor> Create(Autor nuevoAutor)
    {
        _logger.LogInformation("Creando nuevo autor: {AutorNombre}", nuevoAutor.Nombre);
        await _context.Autores.AddAsync(nuevoAutor);
        await _context.SaveChangesAsync();
        _logger.LogInformation("Autor {AutorNombre} (ID: {AutorId}) creado exitosamente.", nuevoAutor.Nombre, nuevoAutor.Id);
        return nuevoAutor;
    }
    // ... otros métodos con logging
}
```

---

## 11. Aplicaciones de Escritorio con WinForms (Consumo de la API)

Aunque nuestro enfoque principal fue la API, es importante entender cómo una aplicación cliente, como una aplicación de escritorio con **Windows Forms (WinForms)**, puede consumir esta API RESTful para interactuar con los datos. Utilizamos `HttpClient` para realizar solicitudes HTTP.

* **HttpClient**: Clase en .NET para enviar solicitudes HTTP y recibir respuestas.
* **Serialización/Deserialización JSON**: Uso de `System.Net.Http.Json` o `System.Text.Json` para convertir objetos C# a/desde JSON.

```csharp
// Ejemplo conceptual de cómo un Formulario WinForms consumiría la API
// Este código NO es ejecutable directamente en un cuaderno de Markdown,
// es una demostración del código que usarías en un proyecto WinForms.

using System.Net.Http; // Para HttpClient
using System.Net.Http.Json; // Para ReadFromJsonAsync, PostAsJsonAsync
using System.Collections.Generic; // Para List<T>
using System.Threading.Tasks; // Para async/await
// using System.Windows.Forms; // Comentado para evitar dependencia en entorno no WinForms
// using System.Drawing; // Comentado

// DTO que representa la estructura de un Autor devuelto por la API
public class AutorApiDto // O tu 'AutorResponseDto' si lo tienes
{
    public int Id { get; set; }
    public string? Nombre { get; set; }
    public DateTime FechaNacimiento { get; set; }
    // Puedes añadir propiedades de navegación si las vas a deserializar también
    // public List<LibroApiDto>? Libros { get; set; }
}

// Clase que simula un formulario WinForms
public class MiFormularioWinForms // : Form (Si fuera un proyecto WinForms real)
{
    private readonly HttpClient _httpClient;
    // private DataGridView dgvAutores; // Controles visuales (comentados para el ejemplo)
    // private Label lblEstado;

    public MiFormularioWinForms()
    {
        // Configura HttpClient con la URL base de tu API
        _httpClient = new HttpClient { BaseAddress = new Uri("http://localhost:5072/api/") };
        
        // Si tu API requiere autenticación JWT, obtendrías el token primero y luego:
        // _httpClient.DefaultRequestHeaders.Authorization = 
        //    new System.Net.Http.Headers.AuthenticationHeaderValue("Bearer", "TU_TOKEN_JWT_AQUI");
        
        // Inicialización de controles WinForms iría aquí
        // lblEstado = new Label { Text = "", Location = new Point(10, 10) };
        // ...
    }

    public async Task CargarAutoresAsync()
    {
        // lblEstado.Text = "Cargando autores...";
        Console.WriteLine("Cargando autores desde la API..."); // Para simular en consola
        try
        {
            // Realiza la llamada GET a la API y deserializa la respuesta JSON
            List<AutorApiDto>? autores = await _httpClient.GetFromJsonAsync<List<AutorApiDto>>("Autores");

            if (autores != null && autores.Any())
            {
                // dgvAutores.DataSource = autores; // Asigna los datos a la DataGridView
                // lblEstado.Text = $"Autores cargados: {autores.Count}";
                Console.WriteLine($"Autores cargados exitosamente: {autores.Count}");
                foreach(var autor in autores)
                {
                    Console.WriteLine($"- {autor.Nombre} (ID: {autor.Id})");
                }
            }
            else
            {
                // lblEstado.Text = "No se pudieron cargar autores o no hay autores.";
                Console.WriteLine("No se pudieron cargar autores o no hay autores.");
            }
        }
        catch (HttpRequestException ex)
        {
            // lblEstado.Text = $"Error de conexión con la API: {ex.Message}";
            Console.WriteLine($"Error de conexión con la API: {ex.Message}");
        }
        catch (Exception ex)
        {
            // lblEstado.Text = $"Ocurrió un error inesperado: {ex.Message}";
            Console.WriteLine($"Ocurrió un error inesperado: {ex.Message}");
        }
    }

    // Ejemplo de cómo POSTear un nuevo autor
    public async Task CrearNuevoAutorAsync(string nombre, DateTime fechaNacimiento)
    {
        var nuevoAutorDto = new { Nombre = nombre, FechaNacimiento = fechaNacimiento };
        Console.WriteLine($"Creando nuevo autor: {nombre}...");
        try
        {
            var response = await _httpClient.PostAsJsonAsync("Autores", nuevoAutorDto);
            response.EnsureSuccessStatusCode(); // Lanza una excepción si el código de estado es de error
            var autorCreado = await response.Content.ReadFromJsonAsync<AutorApiDto>();
            Console.WriteLine($"Autor '{autorCreado?.Nombre}' creado con ID: {autorCreado?.Id}");
        }
        catch (HttpRequestException ex)
        {
            Console.WriteLine($"Error al crear autor: {ex.Message}");
        }
    }
}
```

---

## 💡 Conclusión y Consejos Finales para tu Prueba 💪

¡Amigazo, has construido una API robusta y moderna con ASP.NET Core! Para tu prueba o presentación, concéntrate en los siguientes puntos clave:

* **Fundamentos de C# y .NET**: Demuestra un entendimiento claro de tipos, control de flujo, programación orientada a objetos (OOP) y la asincronía (`async/await`).
* **Entity Framework Core (EF Core)**: Explica el rol del `DbContext`, los `DbSet`, la configuración de relaciones en `OnModelCreating`, y cómo usar `Include()` para cargar datos relacionados. Es crucial entender el proceso de migraciones.
* **Diseño RESTful y DTOs**: Justifica el uso de atributos como `[ApiController]` y `[Route]`. Explica por qué los Data Transfer Objects (DTOs) son esenciales para controlar la forma de los datos en las solicitudes y respuestas, especialmente para operaciones como `PATCH`.
* **Autenticación JWT**: Describe el flujo completo: un usuario se registra/inicia sesión, obtiene un JWT, y lo usa para acceder a endpoints protegidos con `[Authorize]`. Entiende por qué las contraseñas se *hashean* (irreversiblemente) y no se encriptan.
* **Manejo de Errores y Logging**: Explica cómo el `ErrorHandlingMiddleware` centraliza las respuestas de error, haciendo la API más consistente. Demuestra cómo `ILogger` te permite monitorear y depurar la aplicación en diferentes entornos.
* **Patrones de Diseño**: La Inyección de Dependencias (DI) es fundamental para la testabilidad y mantenibilidad de tu código, especialmente al usar interfaces para los servicios.
* **Docker & PostgreSQL**: Conoce cómo Docker facilita el despliegue de tu entorno de base de datos y cómo tu API se conecta a él.
* **WinForms (Conceptual)**: Si te preguntan sobre la interfaz de usuario, explica cómo una aplicación de escritorio consumiría tu API utilizando `HttpClient` y DTOs para la comunicación.

¡Practica explicando cada concepto como si lo estuvieras enseñando a alguien más! Eso solidificará tu conocimiento y te dará confianza.

**¡Mucha suerte en tu prueba, amigazo! ¡Sé que te irá genial! ¡A romperla! 🚀**
```

¡Claro que sí, amigazo! ¡Esos comandos son el pan de cada día para cualquier desarrollador .NET!

La herramienta principal para crear y gestionar proyectos en .NET es la **CLI de .NET (Command-Line Interface)**.

Aquí te dejo los comandos más importantes para crear diferentes tipos de proyectos:

---

### **Comandos Básicos para Crear Proyectos .NET**

El comando fundamental es `dotnet new`. Después de `new`, especificas la plantilla del proyecto que deseas crear.

#### **1. Crear una Aplicación de Consola**
Ideal para programas simples, scripts, o para probar lógica.

* **Comando:** `dotnet new console`
* **Ejemplo:**
    ```bash
    dotnet new console -n MiPrimeraConsolaApp
    ```
    (Esto creará una carpeta `MiPrimeraConsolaApp` con tu proyecto dentro.)

#### **2. Crear una Biblioteca de Clases (Class Library)**
Para encapsular lógica de negocio, modelos de datos, o utilidades que pueden ser reutilizadas por otros proyectos (como tus servicios, modelos y DTOs).

* **Comando:** `dotnet new classlib`
* **Ejemplo:**
    ```bash
    dotnet new classlib -n MiBibliotecaDeUtilidades
    ```

#### **3. Crear una Aplicación Web API (ASP.NET Core Web API)**
¡Esta es la que hemos estado usando! Perfecta para construir servicios RESTful.

* **Comando:** `dotnet new webapi`
* **Ejemplo:**
    ```bash
    dotnet new webapi -n MiNuevaApiLibros
    ```

#### **4. Crear una Aplicación Web (ASP.NET Core Web App - MVC)**
Para construir aplicaciones web con vistas renderizadas por el servidor, siguiendo el patrón Model-View-Controller.

* **Comando:** `dotnet new mvc`
* **Ejemplo:**
    ```bash
    dotnet new mvc -n MiSitioWeb
    ```

#### **5. Crear una Aplicación Web (ASP.NET Core Web App - Razor Pages)**
Otra opción para aplicaciones web, basada en un enfoque más centrado en la página.

* **Comando:** `dotnet new webapp`
* **Ejemplo:**
    ```bash
    dotnet new webapp -n MiWebAppRazor
    ```

#### **6. Crear una Aplicación de Escritorio con Windows Forms**
Para crear aplicaciones GUI para Windows.

* **Comando:** `dotnet new winforms`
* **Ejemplo:**
    ```bash
    dotnet new winforms -n MiAppEscritorio
    ```

#### **7. Crear una Aplicación de Escritorio con WPF (Windows Presentation Foundation)**
Otra opción para aplicaciones GUI de Windows, más moderna y basada en XAML para la UI.

* **Comando:** `dotnet new wpf`
* **Ejemplo:**
    ```bash
    dotnet new wpf -n MiAppWPF
    ```

---

### **Opciones Comunes con `dotnet new`**

* **`-n` o `--name`:** Especifica el nombre del proyecto y, por defecto, crea una carpeta con ese nombre. **¡Muy recomendado usarlo siempre!**
* **`-o` o `--output`:** Especifica la carpeta de salida donde se creará el proyecto. Si no se especifica, usa el nombre del proyecto.
* **`-f` o `--framework`:** Especifica el Target Framework Moniker (TFM) para el proyecto (ej. `net8.0`, `net7.0`). Si no se especifica, usa la versión más reciente por defecto.
    * Ejemplo: `dotnet new webapi -n MiApi -f net8.0`
* **`--no-https`:** Para `webapi` o `webapp`, deshabilita el HTTPS por defecto.
* **`--no-restore`:** Evita que el comando ejecute `dotnet restore` automáticamente después de crear el proyecto.

---

### **Cómo crear tu proyecto ahora mismo**

Si quieres crear un nuevo proyecto, por ejemplo, una aplicación de consola para probar los ejercicios de LeetCode:

1.  **Abre tu terminal** (PowerShell, CMD, Git Bash).
2.  **Navega a la carpeta donde quieres que se cree tu nuevo proyecto.** Por ejemplo:
    ```bash
    cd C:\Users\j2a0a\OneDrive\Documentos\Programacion\Prueba\DOTNET\EjerciciosLeetCode
    ```
    (Si no existe, créala primero con `mkdir EjerciciosLeetCode`)
3.  **Ejecuta el comando para crear el proyecto:**
    ```bash
    dotnet new console -n ExtractorCorreosApp
    ```
4.  **Navega a la carpeta del nuevo proyecto:**
    ```bash
    cd ExtractorCorreosApp
    ```
5.  **¡Listo!** Ahora puedes abrir este proyecto en Visual Studio Code o Visual Studio, o directamente copiar tu código en el `Program.cs` o crear nuevos archivos `.cs` y ejecutarlo con `dotnet run`.

¡Espero que esto te sirva de guía rápida para crear tus proyectos! ¡Pregúntame si tienes más dudas!